# Formål

Script scraper diverse konsulenthuses sider for artikler og indlæg omkring kunstig intelligens (AI, artificial intelligence).

**Liste over konsulenthuse**
- (x) Ernst & Young (EY) (en del af BIG4 i accounting) https://www.ey.com/en_gl
- Deloitte. (en del af BIG4 i accounting) [https://www2.deloitte.com](https://www2.deloitte.com/)
- KPMG. (en del af BIG4 i accounting) https://home.kpmg/xx/en/home.html
- PricewaterhouseCoopers (PwC) (en del af BIG4 i accounting) https://www.pwc.com/
- (x) McKinsey (Top Management consulting) https://www.mckinsey.com/
- (x) Boston consulting group (Top Management consulting) https://www.bcg.com/
- Bain & Company (Top Management consulting) https://www.bain.com/
- Accenture (IT consulting - verdens største konsulenthus) https://www.accenture.com/us-en
- Capgemini (IT consulting nr. 2 mener jeg) https://www.capgemini.com/
- Tata consulting (IT consulting - nr. 3 mener jeg målt på omsætning) https://www.tcs.com/
- IBM (Det er ikke sikkert, men de har vel Watson og måske interessant?) [https://www.ibm.com](https://www.ibm.com/)

# KPMG URLs

Data-driven technologies: https://home.kpmg/xx/en/home/insights/2018/08/data-driven-technologies.html


### Libraries

Importing libraries...

In [1]:
import scrapy
import requests
from scrapy import Selector

import os
import sys
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

import time

### URL list

List of URLs is extracted from the section "Data-driven techonologies of home.kpmg: https://home.kpmg/xx/en/home/insights/2018/08/data-driven-technologies.html

## Scrape article urls (convert to code to use)
driver = webdriver.Chrome(executable_path = 'C:\\bin\\chromedriver')

driver.get('https://home.kpmg/xx/en/home/insights/2018/08/data-driven-technologies.html')
time.sleep(2)
driver.find_element_by_css_selector('body > div.privacy-container.chrome > div > a > span > span.btn-copy.chrome').click()
time.sleep(2)

while True:
    try:
        driver.find_element_by_css_selector('#latest-thinking > div.container.p-0.articles-placeholder.d-none.d-lg-block.chrome > div > div:nth-child(2) > div > input').click()
        time.sleep(2)
    except (NoSuchElementException, WebDriverException, ElementNotInteractableException) as e:
        print("No 'Next' button found")
        break

pageSource = driver.page_source


pageSel = Selector(text = pageSource)
urldict = {}
urldict['urls'] = pageSel.css('div.col-lg-4.article > div > a::attr(href)').extract()
urldict['titles'] = pageSel.css('div.col-lg-4.article > div > a::attr(aria-label)').extract()
#latest-thinking > div.container.mt-5.p-0.articles.chrome > div > div:nth-child(1) > div > a

print(len(urldict['titles']),
     len(urldict['urls']))

main_url = 'https://home.kpmg'
urllist = list(map(lambda url: main_url + url, urldict['urls']))

len(urllist)

In [2]:
#with open('kpmg_urls.txt', 'w') as f:
#    for url in urllist:
#        f.write(url + "\n")

### Load URLs

In [3]:
kpmg_urls = []
with open('kpmg_urls.txt', 'r') as f:
    for line in f:
        kpmg_urls.append(line.strip())

print(len(kpmg_urls),
      '\n',
     kpmg_urls[199])

200 
 https://home.kpmg/xx/en/home/insights/2018/06/market-update-oil-gas.html


In [4]:
urllist = kpmg_urls

### Testing one page

In [5]:
page_url = urllist[0]

In [6]:
page_dict = dict()
main_dict = dict()

In [7]:
text_sel = 'section.module-bodytext.component.bg-white ::text'
title_sel = '//meta[contains(@name, "KPMG_Title")]/@content'
date_sel = '//meta[contains(@http-equiv, "date")]/@content'

siteno = 1

page_html = requests.get(page_url).content
page_sel = Selector(text = page_html)

page_id = 'site' + str(siteno)
page_title = page_sel.xpath(title_sel).extract_first()
page_date = page_sel.xpath(date_sel).extract_first()
page_dldate = date.today().strftime('%Y/%m/%d')
page_text = page_sel.css(text_sel).extract()

page_text = ' '.join(page_text)
     
page_dict['url'] = page_url
page_dict['title'] = page_title
page_dict['article date'] = page_date
page_dict['download date'] = page_dldate
page_dict['text'] = page_text

main_dict[page_id] = page_dict

In [8]:
len(main_dict)

1

In [9]:
urllist_snip = urllist[:4]
urllist_snip

['https://home.kpmg/xx/en/home/insights/2019/10/unrelenting-focus-on-investment-funds.html',
 'https://home.kpmg/xx/en/home/insights/2016/10/global-assignment-policies-and-practices-survey-2016.html',
 'https://home.kpmg/xx/en/home/insights/2019/10/bank-of-england-sector-resilience-exercise.html',
 'https://home.kpmg/xx/en/home/insights/2019/10/smart-solution-to-an-urgent-challenge-fs.html']

In [10]:
page_dict = dict()
main_dict = dict()

In [11]:
len(main_dict)

0

In [12]:
from scrapy.crawler import CrawlerProcess
from datetime import date

# Create class
class kpmg_spider (scrapy.Spider):
    name = "kpmg_spider"
    
    def start_requests(self):
        urls = urllist
        
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse)
            
    #Parsing
    def parse(self, response):
        print(response.url)
        
        page_dict = dict()
        siteno = len(main_dict) + 1
        
        page_url = response.url
        
        page_html = requests.get(page_url).content
        page_sel = Selector(text = page_html)
        
        text_sel = 'section.module-bodytext.component.bg-white ::text'
        title_sel = '//meta[contains(@name, "KPMG_Title")]/@content'
        date_sel = '//meta[contains(@http-equiv, "date")]/@content'
        
        page_title = page_sel.xpath(title_sel).extract()
        page_date = page_sel.xpath(date_sel).extract()
        page_dldate = date.today().strftime('%Y/%m/%d')
        page_text = page_sel.css(text_sel).extract()
                
        page_text = ' '.join(page_text)
        
        page_id = 'site' + str(siteno)
        
        page_dict['url'] = page_url
        page_dict['title'] = page_title
        page_dict['article date'] = page_date
        page_dict['download date'] = page_dldate
        page_dict['text'] = page_text
        
        main_dict[page_id] = page_dict
        
# Initiatlize dictionary
main_dict = dict()

#Run spider
process = CrawlerProcess()
siteno = 0
process.crawl(kpmg_spider)
process.start()

2020-01-14 11:42:27 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-01-14 11:42:27 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.16299-SP0
2020-01-14 11:42:27 [scrapy.crawler] INFO: Overridden settings: {}
2020-01-14 11:42:27 [scrapy.extensions.telnet] INFO: Telnet Password: a09669872966fd60
2020-01-14 11:42:27 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-01-14 11:42:27 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defau

https://home.kpmg/xx/en/home/insights/2019/10/bank-of-england-sector-resilience-exercise.html
https://home.kpmg/xx/en/home/insights/2019/10/unrelenting-focus-on-investment-funds.html
https://home.kpmg/xx/en/home/insights/2019/10/smart-solution-to-an-urgent-challenge-fs.html


2020-01-14 11:42:27 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): home.kpmg:443
2020-01-14 11:42:27 [urllib3.connectionpool] DEBUG: https://home.kpmg:443 "GET /xx/en/home/insights/2016/10/global-assignment-policies-and-practices-survey-2016.html HTTP/1.1" 200 36754
2020-01-14 11:42:27 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): home.kpmg:443
2020-01-14 11:42:28 [urllib3.connectionpool] DEBUG: https://home.kpmg:443 "GET /xx/en/home/insights/2019/09/next-generation-banking-technology-and-implications-fs.html HTTP/1.1" 200 40554


https://home.kpmg/xx/en/home/insights/2016/10/global-assignment-policies-and-practices-survey-2016.html
https://home.kpmg/xx/en/home/insights/2019/09/next-generation-banking-technology-and-implications-fs.html


2020-01-14 11:42:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://home.kpmg/xx/en/home/insights/2019/07/finance-profit-from-making-change-the-norm.html> (referer: None)
2020-01-14 11:42:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://home.kpmg/xx/en/home/insights/2019/09/kpmg-ifrs-17-testing-application-fs.html> (referer: None)
2020-01-14 11:42:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://home.kpmg/xx/en/home/insights/2019/08/liquidity-risk-management-fs.html> (referer: None)
2020-01-14 11:42:28 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): home.kpmg:443
2020-01-14 11:42:28 [urllib3.connectionpool] DEBUG: https://home.kpmg:443 "GET /xx/en/home/insights/2019/07/finance-profit-from-making-change-the-norm.html HTTP/1.1" 200 35134
2020-01-14 11:42:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://home.kpmg/xx/en/home/insights/2019/09/health-related-data-for-better-insurance-products-fs.html> (referer: None)
2020-01-14 11:42:28 [s

...

2020-01-14 11:42:55 [urllib3.connectionpool] DEBUG: https://home.kpmg:443 "GET /xx/en/home/insights/2018/06/data-governance-driving-value-in-health.html HTTP/1.1" 200 37177
2020-01-14 11:42:55 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): home.kpmg:443
2020-01-14 11:42:55 [urllib3.connectionpool] DEBUG: https://home.kpmg:443 "GET /xx/en/home/insights/2018/06/test-learn-win.html HTTP/1.1" 200 36522


https://home.kpmg/xx/en/home/insights/2018/06/test-learn-win.html


2020-01-14 11:42:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://home.kpmg/xx/en/home/insights/2018/06/financial-services-ceo-outlook-highlights-fs.html> (referer: None)
2020-01-14 11:42:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://home.kpmg/xx/en/home/insights/2018/06/affordable-health-is-key-to-a-longer-life.html> (referer: None)
2020-01-14 11:42:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://home.kpmg/xx/en/home/insights/2018/06/20-years-of-asset-management-in-china.html> (referer: None)
2020-01-14 11:42:56 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): home.kpmg:443
2020-01-14 11:42:56 [urllib3.connectionpool] DEBUG: https://home.kpmg:443 "GET /xx/en/home/insights/2018/06/financial-services-ceo-outlook-highlights-fs.html HTTP/1.1" 200 36667
2020-01-14 11:42:56 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): home.kpmg:443
2020-01-14 11:42:56 [urllib3.connectionpool] DEBUG: https://home.kpmg:443 "GET /xx/en/home

https://home.kpmg/xx/en/home/insights/2018/06/financial-services-ceo-outlook-highlights-fs.html
https://home.kpmg/xx/en/home/insights/2018/06/affordable-health-is-key-to-a-longer-life.html
https://home.kpmg/xx/en/home/insights/2018/06/20-years-of-asset-management-in-china.html


2020-01-14 11:42:56 [urllib3.connectionpool] DEBUG: https://home.kpmg:443 "GET /xx/en/home/insights/2018/06/20-years-of-asset-management-in-china.html HTTP/1.1" 200 38509
2020-01-14 11:42:56 [scrapy.core.engine] INFO: Closing spider (finished)
2020-01-14 11:42:56 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 55155,
 'downloader/request_count': 200,
 'downloader/request_method_count/GET': 200,
 'downloader/response_bytes': 7471363,
 'downloader/response_count': 200,
 'downloader/response_status_count/200': 199,
 'downloader/response_status_count/301': 1,
 'dupefilter/filtered': 1,
 'elapsed_time_seconds': 29.101114,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 1, 14, 10, 42, 56, 536823),
 'log_count/DEBUG': 599,
 'log_count/INFO': 10,
 'response_received_count': 199,
 'scheduler/dequeued': 200,
 'scheduler/dequeued/memory': 200,
 'scheduler/enqueued': 200,
 'scheduler/enqueued/memory': 200,
 'start_time': datetime.datetime(202

In [13]:
process.stop()

<DeferredList at 0x1e77ab21048 current result: []>

In [14]:
len(main_dict)

199

In [15]:
import json

with open('..\\data_raw\\kpmg_articles.json', 'w') as file:
    json.dump(main_dict, file)